In [ ]:
#!/usr/bin/env python
# coding: utf-8

In [ ]:
import codecs
from datetime import datetime as dt
import json
import sys
import numpy as np
import os
import pandas as pd
import plotly
from plotly import subplots
import plotly.express as px
import plotly.tools as tls
import plotly.graph_objects as go
import plotly.io as pio
import plotly.offline as offline
import sys
if "ipy" in sys.argv[0]:
    offline.init_notebook_mode()
from cov19utils import create_basic_plot_figure, \
    show_and_clear, moving_average, \
    blank2zero, csv2array, \
    get_twitter, tweet_with_image, \
    get_gpr_predict, FONT_NAME, DT_OFFSET, \
    download_if_needed, json2nparr, code2int, age2int, \
    get_populations, get_os_idx_of_arr, dump_val_in_arr, \
    calc_last1w2w_dif, create_basic_scatter_figure, \
    show_and_save_plotly
import re
import requests
from sklearn.linear_model import LinearRegression

In [ ]:
curr = pd.read_csv("csv/sapporo.csv", encoding='shift-jis', header=0)
n = len(curr)
print(n)

In [ ]:
prev = 0
with codecs.open("csv/sapporo.tmp", encoding="utf-8") as f:
    prev = f.read().strip()
prev = int(prev)
print(prev)

In [ ]:
if n == prev:
    print("maybe the same data, nothing to do.")
    if "ipy" in sys.argv[0]:
        pass#exit()
    else:
        sys.exit()

In [ ]:
curr['untrack'] = ((curr['postive'] - curr['trace']) / curr['postive']) * 100
curr['posrate'] = (curr['postive'] / curr['test']) * 100

In [ ]:
df = curr

In [ ]:
titles = ["検査数","陽性者","経路不明率","陽性率"]
cols = ['test', 'postive', 'untrack', 'posrate']
fig = subplots.make_subplots(shared_xaxes=True,
    rows=4, cols=1, subplot_titles=titles,
    horizontal_spacing=0.03, vertical_spacing=0.07)
i = 0
for c in cols:
    trace = go.Bar(x=df['date'], y=df[c], name=titles[i])
    fig.add_trace(trace, 1 + i, 1)
    i += 1

today_str = dt.now().isoformat()[:16].replace('T', ' ')
title = '札幌感染状況 {}'.format(today_str)
fig.update_layout(
    template='plotly_dark',
    margin={"r":10,"t":80,"l":10,"b":100},
    height=700, showlegend=False,
    title=title)
show_and_save_plotly(fig, "sapporo.jpg", js=False, show=True, image=True, html=True)

In [ ]:
tw = get_twitter()
tweet_with_image(tw, "docs/images/sapporo.jpg", title)

In [ ]:
with open("csv/sapporo.tmp", "w") as f:
    f.write("{}".format(n))